In [2]:
### 5.4.1 不带参数的层
import torch
import torch.nn.functional as F
import torch.nn as nn

class CenteredLayer(torch.nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, X):
        return X - X.mean()
layer = CenteredLayer()
layer(torch.FloatTensor([1, 3, 3, 5, 8]))

tensor([-3., -1., -1.,  1.,  4.])

In [3]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [4]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(1.6298e-09, grad_fn=<MeanBackward0>)

In [5]:
###5.4.2 带参数的层
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [6]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-2.1969, -0.9032,  0.0677],
        [ 1.0362,  0.3927, -1.2357],
        [ 2.0910,  1.5024, -1.3546],
        [-0.7833,  0.6041,  0.3980],
        [ 0.9324, -0.4150,  0.5066]], requires_grad=True)

In [7]:
linear(torch.rand(2, 5))

tensor([[0.1284, 0.4850, 0.0000],
        [0.1152, 0.9498, 0.0000]])

In [8]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.0000],
        [1.3918]])

1.设计一个接受输入并计算张量降维的层，它返回$y_k = \sum_{i, j} W_{ijk} x_i x_j$

In [9]:
import torch
import torch.nn as nn

class TensorReductionLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(TensorReductionLayer, self).__init__()
        # 初始化权重张量 W，形状为 (output_dim, input_dim, input_dim)
        self.W = nn.Parameter(torch.randn(output_dim, input_dim, input_dim))

    def forward(self, x):
        # x 的形状为 (batch_size, input_dim)
        batch_size, input_dim = x.size()
        # 计算 y_k = sum_{i, j} W_{ijk} x_i x_j
        y = torch.einsum('bij,bj->bi', torch.einsum('ijk,bj->bik', self.W, x), x)
        return y

# 示例用法
input_dim = 4
output_dim = 3
layer = TensorReductionLayer(input_dim, output_dim)
x = torch.randn(5, input_dim)  # 假设 batch_size 为 5
y = layer(x)
print(y)

tensor([[  0.6070,   4.3160,   2.0065],
        [ -5.9531,   6.0379,  11.3702],
        [ -1.4948,  -2.2645,  -0.0199],
        [-10.7556,  -0.4294,   3.4145],
        [  0.1287,  -0.1465,   0.3475]], grad_fn=<ViewBackward0>)


2.设计一个返回输入数据的傅立叶系数前半部分的层。

In [10]:
import torch
import torch.nn as nn

class FourierTransformLayer(nn.Module):
    def __init__(self):
        super(FourierTransformLayer, self).__init__()

    def forward(self, x):
        # 计算傅立叶变换
        x_fft = torch.fft.fft(x, dim=-1)
        # 只取前半部分系数
        x_fft_half = x_fft[..., :x_fft.size(-1) // 2]
        return x_fft_half

# 示例用法
layer = FourierTransformLayer()
x = torch.randn(5, 8)  # 假设输入形状为 (batch_size, input_dim)
y = layer(x)
print(y)

tensor([[ 2.7904+0.0000j,  3.2212-1.2828j,  1.9614-2.3039j,  0.4601+1.5165j],
        [ 1.4940+0.0000j, -0.9626-0.9018j, -1.8763+1.9419j, -0.2853+1.0206j],
        [ 2.0841+0.0000j, -0.2409-1.0400j,  0.3734-2.3279j,  1.0405-3.3100j],
        [ 1.6647+0.0000j,  0.7294+2.2995j,  0.4377+4.2767j, -0.6177+0.6312j],
        [-1.2008+0.0000j, -0.2110-3.3579j,  2.3750-2.1237j, -3.1509-2.1700j]])
